# SQL agent

In [1]:
import db_connect 

from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain import agents

from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain.agents.agent_toolkits.sql import base

### Load models

In [2]:
llama3 = Ollama(model="llama3:8b", temperature=0)
llama3_embeddings = OllamaEmbeddings(model="llama3:8b", temperature=0)

llama3_inst = Ollama(model="llama3:instruct", temperature=0)
llama3_inst_q8 = Ollama(model="llama3:8b-instruct-q8_0", temperature=0)
llama3_inst_fp16 = Ollama(model="llama3:8b-instruct-fp16", temperature=0)

### Connect to DB with Readonly role

In [3]:
db = db_connect.get_db()

#### Check connection

In [4]:
db.run("select * from passenger")

"[(16, 'John'), (17, 'James'), (18, 'Poul'), (19, 'Christofer'), (20, 'Superman')]"

-----

In [5]:
prefix =\
"""# Personality
You are an expert in PostgreSQL. Answer a question from an ordinary employee.
# Task
Please answer the following question to the best of your ability.
# Tools
Use the following tools to answer:"""

format_inst =\
"""#Format
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question"""

suffix =\
"""# Start chain of thoughts

Question: {input}
Thought:{agent_scratchpad}"""

err_handle =\
"""# Error
Please check your answer for these common mistakes:
- There are both an answer and an action in the response.
- The answer is not final.
- The response is not based on the data from the database query."""

agent = base.create_sql_agent(
    llm=llama3_inst_fp16,
    db=db,
    agent_type=agents.AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    prefix=prefix,
    format_instructions=format_inst,
    suffix=suffix,
    verbose=True,
    agent_executor_kwargs={
        "handle_parsing_errors": err_handle
    }
)

In [6]:
res = agent.invoke({
    "input": "Select the names of all the people who are in the airline database"
})



> Entering new SQL Agent Executor chain...
Let's start by listing the tables in the airline database.

Action: sql_db_list_tables
Action Input: (empty stringcompany, pass_in_trip, passenger, trip

In [ ]:
print(res)

{'input': 'Select the names of all the people who are in the airline database', 'output': 'Agent stopped due to iteration limit or time limit.'}


In [ ]:
print(res['output'])

Agent stopped due to iteration limit or time limit.
